<a href="https://colab.research.google.com/github/dayaneaguiaramos/Estudo_NDVI_Pivos/blob/main/Case_Pivos_atualizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Monitoramento de Indices de Vegetação com o Google Earth Engine

1- Desenvolver um script em Python utilizando a API do Google Earth Engine para monitorar e analisar índices de vegetação (NDVI) em uma área agrícola específica ao longo de um período de tempo.. Utilizando imagens de satélite, você precisa criar um algoritmo que calcule o Índice de Vegetação por Diferença Normalizada (NDVI) para uma área determinada e analise as mudanças ao longo de um período de um ano.

In [ ]:
# 1 Configurando o Ambiente (Instalando e importanto as Bibliotecas)
# 2 Escolher uma área agrícola para o estudo
# 3 Importando os dados de satelite (imagens do Sentinel-2) para um periodo de um ano
# 4 Desenvolver, algoritmo para calcular o NDVI utilizando as bandas de infravermelho próximo (NIR) e vermelho (RED) das imagens de satélite.
# 5 Aplicar o algoritmo às imagens importadas para gerar um série temporal de NDVI
# 6 Analisar a série temporal de NDVI para identificar tendencias e padrões na saude das plantações ao longo do ano
# 7 Gerar gráficos que ilustrem a variação do NDVI ao longo do tempo
# 8 Criar visualizações interativas utilizando a biblioteca geemap para mostrar a área de estudo e a variação do NDVI
# 9 Exportar os resultados (gráficos e dados) para arquivos que possam ser compartilhados com a equipe de tomada de decisão.



 Caso de Estudo - Área agricula no municipio de Paracatu



In [ ]:
# Instalando Bibliotecas necessarias
!pip install matplotlib -q
!pip install numpy
!pip install geemap -q
!pip install geopandas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.4 MB/s eta 0:00:00


In [ ]:
# Importando as Bibliotecas (GEE, matplotlib, numpy, pandas)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import geemap
import ee
import json

In [ ]:
# Inicializando o API do GEE
ee.Authenticate()
# Inicialize o API do GEE
ee.Initialize(project="ee-projetomivegetacao")


In [ ]:
#  Montando um acessando para o google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Lendo o arquivo shapefile que criei para a area de estudo
shapefile_path=("/content/drive/MyDrive/Colab Notebooks/Case_NDVI/Shapefile /Pivos_Paracatu.shp")
gdf=gpd.read_file(shapefile_path)

gdf

,id,Nome,geometry
0,1,area_de_estudo,"POLYGON ((-46.97244 -17.27223, -46.97128 -17.2..."


GEE não permite a utilização direta de um arquivo shp. Converte a base para json


In [ ]:
# Convertendo o shp para to json
gdf = gdf.to_json ()
# Carregando o arquivo json
gdf=json.loads(gdf)
# Selecionando as features
gdf=gdf["features"]
gdf


[{'id': '0',
  'type': 'Feature',
  'properties': {'id': 1, 'Nome': 'area_de_estudo'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-46.97243941554349, -17.272226821788372],
     [-46.971283424180015, -17.273170119408007],
     [-46.96971232682695, -17.27583440734657],
     [-46.96935502040553, -17.275734058216628],
     [-46.96822004706686, -17.274931263209343],
     [-46.96691692952985, -17.274509794430184],
     [-46.966160280637396, -17.274389374601913],
     [-46.966013154463866, -17.274208744711927],
     [-46.96534057767058, -17.274208744711927],
     [-46.96445782062938, -17.274429514553418],
     [-46.9637432077865, -17.274871053442777],
     [-46.962986558894045, -17.27541294063349],
     [-46.962377036175134, -17.275713988384084],
     [-46.96193565765453, -17.276195663761786],
     [-46.96084272036544, -17.277078732016673],
     [-46.959812837150714, -17.27808221353158],
     [-46.95903517023347, -17.279888466482376],
     [-46.958867026035136, -17.280289853621525

In [ ]:
# Defininado filtros na feature
area_de_estudo = ee.FeatureCollection(gdf)

In [ ]:
# Obtendo um ee.Imagem de um ee.ImageCollection
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                                .filterBounds(area_de_estudo)\
                                .filterDate('2023-01-01', '2023-12-31')\
                                .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))\
                                .sort("CLOUDY_PIXEL_PERCENTAGE")

#Print para saber a quantidade das imagens
print("Qtd", image.size().getInfo())


Qtd 40


In [ ]:
# Peguei uma imagem com a menor % de nuvens
image_cloud = image.first()

In [ ]:
# Lista de todas as propriedades de metadados.
print(image_cloud.propertyNames().getInfo())

['system:version', 'system:id', 'SPACECRAFT_NAME', 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE', 'BOA_ADD_OFFSET_B12', 'CLOUD_SHADOW_PERCENTAGE', 'system:footprint', 'SENSOR_QUALITY', 'GENERATION_TIME', 'CLOUDY_PIXEL_OVER_LAND_PERCENTAGE', 'CLOUD_COVERAGE_ASSESSMENT', 'THIN_CIRRUS_PERCENTAGE', 'GRANULE_MEAN_WV', 'BOA_ADD_OFFSET_B1', 'BOA_ADD_OFFSET_B2', 'DATASTRIP_ID', 'BOA_ADD_OFFSET_B5', 'BOA_ADD_OFFSET_B6', 'BOA_ADD_OFFSET_B3', 'BOA_ADD_OFFSET_B4', 'BOA_ADD_OFFSET_B9', 'BOA_ADD_OFFSET_B7', 'BOA_ADD_OFFSET_B8', 'GRANULE_ID', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8', 'DATATAKE_TYPE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4', 'NOT_VEGETATED_PERCENTAGE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B5', 'RADIOMETRIC_QUALITY', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1', 'HIGH_PROBA_CLOUDS_PERCENTAGE', 'UNCLASSIFIED_PERCENTAGE', 'OZONE_SOURCE', 'GRANULE_MEAN_A

In [ ]:
# Conhecendo o ID da imagem
print(image_cloud.get("system:id").getInfo())

COPERNICUS/S2_SR_HARMONIZED/20231215T132229_20231215T132231_T23KKA


In [ ]:
# Extraindo uma propriedade especifica - Porcentagem de Nuvens
print(image_cloud.get("CLOUDY_PIXEL_PERCENTAGE").getInfo())

0.000557


In [ ]:
# Obtendo informações projeção da banda 8
projection = image_cloud.select("B8").projection()
print(projection.getInfo())

{'type': 'Projection', 'crs': 'EPSG:32723', 'transform': [10, 0, 199980, 0, -10, 8100040]}


In [ ]:
# Obtendo informações de escala (em metros) da banda 8
print("escala", projection.nominalScale().getInfo())

escala 10


In [ ]:
# Obtendo informações projeção da banda 4
projection = image_cloud.select("B4").projection()
print(projection.getInfo())


{'type': 'Projection', 'crs': 'EPSG:32723', 'transform': [10, 0, 199980, 0, -10, 8100040]}


In [ ]:
# Obtendo informações de escala (em metros) da banda 4
print("escala", projection.nominalScale().getInfo())

escala 10


In [ ]:
# Mapa de localização para conhecimento da imagem

map=geemap.Map()
map.centerObject(area_de_estudo,10)
map.add_basemap("HYBRID")
map.addLayer(image_cloud.clip(area_de_estudo),{"bands":["B4","B3","B2"], "min":0, "max":2839},"Imagem RGB")
limite=ee.Image().paint(area_de_estudo,0,2)
map.addLayer(limite,{"palette":"red"},"ROI")
display(map)


Map(center=[-17.309378329877102, -46.9873722026027], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Calculo do NDVI da Imagem

nir="B8"
red="B4"
ndvi=image_cloud.expression("(NIR-RED)/(NIR+RED)",{
    "NIR":image_cloud.select(nir),
    "RED":image_cloud.select(red)
})
map.addLayer(ndvi.clip(area_de_estudo),{"palette":["red","white","green"]},"NDVI")
display(map)


Map(bottom=144173.0, center=[-17.309378329876807, -46.987372202603105], controls=(WidgetControl(options=['posi…

In [ ]:
outfp="base_map.html"
# Salvando o mapa
map.save(outfp)



In [ ]:
# Instalaçao da Biblioteca Altair (visualização de dados)
!pip install altair -q

In [ ]:
# Importando as bibliotecas necessarias
import json
import geemap
# Importa a Biloteca geemap (grafico) e Autair (alt)
import geemap.colormaps as cm
import altair as alt


In [ ]:
# Criando uma lista para armazenar os valores do NDVI ao longo do tempo.
ndvi_values=[]

# Iterar sobre todas as imagens da coleção
image_list=image.toList(image.size())

# Para obter o NDVI
for i in range(image_list.length().getInfo()):

  # Obter a imagem pelo indice da imagem
  img=ee.Image(image_list.get(i))

  #Calculo do NDVI
  ndvi_img=img.normalizedDifference(["B8","B4"])

  # Obter as datas da imagem
  date=ee.Date(img.get("system:time_start")).format("YYYY-MM-dd").getInfo()

  # Reduzir a região para obter a media (NDVI) para a area de estudo
  ndvi_value=ndvi_img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=area_de_estudo,
      scale=10).get("nd")
  # Adiciona a data e o valor medio do NDVI para a lista
  ndvi_values.append({"date":date,"ndvi":ndvi_value.getInfo()})

# Converte a lista em Dataframe Panda(pd)
df=pd.DataFrame(ndvi_values)

# Converte coluna de Datetime
df["date"]=pd.to_datetime(df["date"])

# Ordenar o Dataframe pelo coluna date
df=df.sort_values(by="date")

# Exibe o Dataframe com os valores do NDVI ao longo do tempo
print(df)


         date      ndvi
32 2023-02-03  0.655276
39 2023-02-08  0.605952
1  2023-03-25  0.468597
19 2023-03-30  0.463988
34 2023-04-24  0.470197
30 2023-04-29  0.542115
6  2023-05-04  0.547932
3  2023-05-14  0.554669
2  2023-05-19  0.554218
21 2023-05-24  0.526428
27 2023-05-29  0.513575
8  2023-06-03  0.515507
22 2023-06-08  0.494138
24 2023-06-13  0.465741
9  2023-06-18  0.475534
18 2023-06-23  0.454175
5  2023-06-28  0.440349
13 2023-07-03  0.421819
25 2023-07-08  0.400424
14 2023-07-13  0.376906
16 2023-07-23  0.359754
36 2023-07-28  0.336440
11 2023-08-02  0.329552
4  2023-08-07  0.316593
29 2023-08-12  0.294365
28 2023-08-17  0.258606
7  2023-08-22  0.287490
10 2023-09-01  0.254936
12 2023-09-11  0.290550
23 2023-09-16  0.277579
20 2023-09-21  0.270428
31 2023-09-26  0.254381
38 2023-10-16  0.267375
35 2023-10-21  0.279191
26 2023-11-05  0.335765
17 2023-11-10  0.331424
33 2023-11-15  0.316136
15 2023-12-10  0.431109
0  2023-12-15  0.465360
37 2023-12-25  0.432418


In [ ]:
# Iterar sobre todas as imagens da coleção
image_list=image.toList(image.size())

# Para obter o NDVI a Media Min
for i in range(image_list.length().getInfo()):

  # Obter a imagem pelo indice da imagem
  img=ee.Image(image_list.get(i))

  #Calculo do NDVI
  ndvi_img=img.normalizedDifference(["B8","B4"])

  # Obter as datas da imagem
  date=ee.Date(img.get("system:time_start")).format("YYYY-MM-dd").getInfo()

  # Reduzir a região para obter a media (NDVI) para a area de estudo
  ndvi_value=ndvi_img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=area_de_estudo,
      scale=10).get("nd")
  # Adiciona a data e o valor medio do NDVI para a lista
  ndvi_values.append({"date":date,"ndvi":ndvi_value.getInfo()})

# Converte a lista em Dataframe Panda(pd)
df_min=pd.DataFrame(ndvi_values)

# Converte coluna de Datetime
df_min["date"]=pd.to_datetime(df["date"])

# Ordenar o Dataframe pelo coluna date
df_min=df_min.sort_values(by="date")

# Exibe o Dataframe com os valores do NDVI ao longo do tempo
print(df_min)

         date      ndvi
32 2023-02-03  0.655276
39 2023-02-08  0.605952
1  2023-03-25  0.468597
19 2023-03-30  0.463988
34 2023-04-24  0.470197
..        ...       ...
75        NaT  0.279191
76        NaT  0.336440
77        NaT  0.432418
78        NaT  0.267375
79        NaT  0.605952

[80 rows x 2 columns]


In [ ]:
# Iterar sobre todas as imagens da coleção
image_list=image.toList(image.size())

# Para obter o NDVI a Media Min
for i in range(image_list.length().getInfo()):

  # Obter a imagem pelo indice da imagem
  img=ee.Image(image_list.get(i))

  #Calculo do NDVI
  ndvi_img=img.normalizedDifference(["B8","B4"])

  # Obter as datas da imagem
  date=ee.Date(img.get("system:time_start")).format("YYYY-MM-dd").getInfo()

  # Reduzir a região para obter a media (NDVI) para a area de estudo
  ndvi_value=ndvi_img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=area_de_estudo,
      scale=10).get("nd")
  # Adiciona a data e o valor medio do NDVI para a lista
  ndvi_values.append({"date":date,"ndvi":ndvi_value.getInfo()})

# Converte a lista em Dataframe Panda(pd)
df_max=pd.DataFrame(ndvi_values)

# Converte coluna de Datetime
df_max["date"]=pd.to_datetime(df["date"])

# Ordenar o Dataframe pelo coluna date
df_max=df_max.sort_values(by="date")

# Exibe o Dataframe com os valores do NDVI ao longo do tempo
print(df_max)

          date      ndvi
32  2023-02-03  0.655276
39  2023-02-08  0.605952
1   2023-03-25  0.468597
19  2023-03-30  0.463988
34  2023-04-24  0.470197
..         ...       ...
115        NaT  0.279191
116        NaT  0.336440
117        NaT  0.432418
118        NaT  0.267375
119        NaT  0.605952

[120 rows x 2 columns]


In [ ]:
# Combinar os Dataframe (medio,minimo e maximo)
df["type"]="NDVI Medio"
df_min["type"]="NDVI Minimo"
df_max["type"]="NDVI Maximo"

# Renomear a coluna de NDVI
df=df.rename(columns={"ndvi":"NDVI"})
df_min=df_min.rename(columns={"ndvi":"NDVI"})
df_max=df_max.rename(columns={"ndvi":"NDVI"})
# Combina os Dataframe
df_combined=pd.concat([df,df_min,df_max])

print(df_combined)


          date      NDVI         type
32  2023-02-03  0.655276   NDVI Medio
39  2023-02-08  0.605952   NDVI Medio
1   2023-03-25  0.468597   NDVI Medio
19  2023-03-30  0.463988   NDVI Medio
34  2023-04-24  0.470197   NDVI Medio
..         ...       ...          ...
115        NaT  0.279191  NDVI Maximo
116        NaT  0.336440  NDVI Maximo
117        NaT  0.432418  NDVI Maximo
118        NaT  0.267375  NDVI Maximo
119        NaT  0.605952  NDVI Maximo

[240 rows x 3 columns]


In [ ]:
# Criando o  Graficos (barras) para o NDVI minimo
barras_min=alt.Chart(df_combined[df_combined["type"]=="NDVI Minimo"]).mark_bar(size=6,color="red").encode(


    x=alt.X("date:T",title="data"),
    y=alt.Y("NDVI:Q",title="NDVI"),
    color=alt.Color("type:N",legend=alt.Legend(title="Tipo de NDVI")),
    tooltip=[
        alt.Tooltip("date:T",title="DATA"),
        alt.Tooltip("NDVI:Q",title="NDVI")
    ]
)


# Criando o  Graficos (barras) para o NDVI maximo
barras_max=alt.Chart(df_combined[df_combined["type"]=="NDVI Maximo"]).mark_bar(size=6,color="blue").encode(


    x=alt.X("date:T",title="data"),
    y=alt.Y("NDVI:Q",title="NDVI"),
    color=alt.Color("type:N",legend=alt.Legend(title="Tipo de NDVI")),
    tooltip=[
        alt.Tooltip("date:T",title="DATA"),
        alt.Tooltip("NDVI:Q",title="NDVI")
    ]
)

# Criando o  Graficos (linha) para o NDVI media
linha_men=alt.Chart(df_combined[df_combined["type"]=="NDVI Medio"]).mark_line(size=2,color="orange").encode(

    x=alt.X("date:T",title="data"),
    y=alt.Y("NDVI:Q",title="NDVI"),
    color=alt.Color("type:N",legend=alt.Legend(title="Tipo de NDVI")),
    tooltip=[
        alt.Tooltip("date:T",title="DATA"),
        alt.Tooltip("NDVI:Q",title="NDVI")
    ]
)

# Combinar os graficos
comb_grafico=alt.layer(barras_min,barras_max,linha_men).properties(
    title="Serie Historica do NDVI para o ano de 2023",width=1000,height=300
)

# Exibindo o grafico
comb_grafico.display()

alt.LayerChart(...)